In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
import os
# using python csv library for writing results
import csv
# loading data in windows environment
import pathlib
import re
print (tf.__version__)
from kaggle_datasets import KaggleDatasets
AUTO = tf.data.experimental.AUTOTUNE
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# Loading Test Files from the Competition Dataset
TEST_NAMES = tf.io.gfile.glob("../input/cassava-leaf-disease-classification/test_tfrecords/*.tfrec")

# Helper Function
def mapping_image_name(dat):
    # Extracting Image Content and Name from dataset
    TFREC_MAP = {"image": tf.io.FixedLenFeature([], tf.string),
                 "image_name": tf.io.FixedLenFeature([], tf.string),
                }
    # Parsing the Extracted Data
    parsed_dat = tf.io.parse_single_example(dat, TFREC_MAP)
    # Decoding Image from Extracted Data
    image = tf.io.decode_jpeg(parsed_dat['image'], channels=3)
    # Normalizing the Image Data
    image = tf.cast(image/255, dtype=tf.float32)
    # Loading the Image Target Label
    target = parsed_dat['image_name']
    # Returning the tuple
    return image, target

# Define the Image Height
IMG_HEIGHT = 386
# Define the Image Width
IMG_WIDTH = 386

T_NAMES=[]
IMG=[]

# Loading the Dataset
TEST_DATA = tf.data.TFRecordDataset(TEST_NAMES, num_parallel_reads=AUTO)
# Loading the Test Data
T_DATA = TEST_DATA.map(mapping_image_name)

# Define the Data Augmentation
data_augmentation = tf.keras.Sequential([
  # Random Flip the Image
  tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  # Random Rotating the Image
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
  #tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
])

# Helper Function for Extracting Image Content
def mapping_image(dat):
    TFREC_MAP = {"image": tf.io.FixedLenFeature([], tf.string)}
    parsed_dat = tf.io.parse_single_example(dat, TFREC_MAP)
    image = tf.io.decode_image(parsed_dat['image'], channels=3)
    image = tf.cast(image/255, dtype=tf.float32)
    return image

# Helper Function for Extracting Image ID
def mapping_id(dat):
    TFREC_MAP = {"target": tf.io.FixedLenFeature([], tf.int64)}
    parsed_dat = tf.io.parse_single_example(dat, TFREC_MAP)
    image_id = parsed_dat['target']
    image_id = tf.cast(image_id, dtype=tf.int32)
    return image_id

# Helper Function for Extracting Image Name
def mapping_name(dat):
    TFREC_MAP = {"image_name": tf.io.FixedLenFeature([], tf.string)}
    parsed_dat = tf.io.parse_single_example(dat, TFREC_MAP)
    image_name = parsed_dat['image_name']
    return image_name



import os

# Results list for submission
RESULTS = []

# Getting predictions

# Loading the Pretrained Model from Dataset
# The pretrained model code is defined in another file
kaggle_model = tf.keras.models.load_model('../input/cassava/kaggle_cassava_model_tpu_V20.h5')

# Loading the Image and Label for Prediction
for image, label in T_DATA:
    # Resized the Testing Image
    re_image = tf.image.resize_with_pad(image, IMG_HEIGHT, IMG_WIDTH, method=tf.image.ResizeMethod.BICUBIC, antialias=False)
    # Load Image into List
    IMG.append(re_image)
    # Convert List into Tensor Object
    img = tf.convert_to_tensor(IMG, dtype=tf.float32)
    # Checking the Image Shape
    print (img.shape)
    # Using the Model to predict
    predict = kaggle_model.predict(img)
    # Loading the Maximum Probability
    V = tf.math.argmax(predict[0]).numpy()
    print (V)
    # Loading the result into list
    RESULTS.append(V)
    # Loading Image Name
    T_NAMES.append(label.numpy().decode('utf-8'))
    # Clear Image List
    IMG=[]

#preds = kaggle_model.predict(DATA)

#preds = model.predict(IMG[0])

#for dat in preds:
    #print (dat)
    #V = tf.math.argmax(dat).numpy()
    #RESULTS.append(V)

# Checking the Prediction Results
print (RESULTS)
# Checking the Image Names
print (T_NAMES)

# Prediction Submission
np.savetxt(
    'submission.csv',
    np.rec.fromarrays([T_NAMES, RESULTS]),
    fmt=['%s', '%d'],
    delimiter=',',
    header='image_id,label',
    comments='',
)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session